In [ ]:
!git clone https://github.com/imvladikon/huawei-nlpcourse-project

fatal: destination path 'huawei-nlpcourse-project' already exists and is not an empty directory.


In [ ]:
import os
os.chdir("huawei-nlpcourse-project")

In [ ]:
%%capture

!pip install -r requirements.txt
!pip install lda

In [ ]:
import gensim.corpora as corpora
from utils.preprocessing import Vocab, preprocess_text, HebTokenizer

from sklearn.metrics import *
import numpy as np
np.random.seed(42)
import random
random.seed(42)

import pandas as pd
from sklearn.model_selection import train_test_split
from utils.string_utils import preprocessing
from tqdm.notebook import tqdm

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/hebert/hebrew_news_general_category.csv")
df["row_num"] = df.index
df = df[~df.label_new.isna()]
df["label"] = df["label_new"]

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,11,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2)
# df_val, df_test = train_test_split(df_train, test_size=0.5)

# Preprocess texts

In [ ]:
tqdm._instances.clear()

In [ ]:
texts_train = preprocess_text(df_train.articleBody.values)

text preprocessing: 100%|██████████| 36949/36949 [00:57<00:00, 643.75it/s]


In [ ]:
texts_test = preprocess_text(df_test.articleBody.values)

text preprocessing: 100%|██████████| 9238/9238 [00:15<00:00, 612.43it/s]


In [ ]:
vocab_file = 'data/vocab.txt'

In [ ]:
vocab_cls = Vocab(texts_train).save(vocab_file)

to file: 100%|██████████| 50000/50000 [00:00<00:00, 1143647.41it/s]


In [ ]:
word2id = vocab_cls.w2id
id2word = vocab_cls.id2w
vocab = vocab_cls.sorted_word_counts
word2freq = vocab_cls.word2freq

No significant differences with and without <'num'> token

In [ ]:
indexed_texts = vocab_cls.texts_to_ids(texts_train)

In [ ]:
indexed_test = vocab_cls.texts_to_ids(texts_test)

In [ ]:
list(df.label.unique())

['sports', 'tech', 'politics', 'health', 'entertainment', 'food', 'criminal']

In [ ]:
labels = df.label.values

In [ ]:
cls2id = {'sports':1,'tech':2,'politics':3,'health':4,'entertainment':5,'food':6,'criminal':7}

In [ ]:
train_labels = df_train.label.map(cls2id)
test_labels = df_test.label.map(cls2id)

In [ ]:
id2word_corpora = corpora.Dictionary(texts_train, prune_at=10000)
corpus = [id2word_corpora.doc2bow(text) for text in texts_train]

In [ ]:
corpus_test = [id2word_corpora.doc2bow(text) for text in texts_test]

In [ ]:
def preprocess_text(texts):
    heb_tokenizer = HebTokenizer()

    result = []
    for line in tqdm(texts, desc="text preprocessing"):
        tokens = heb_tokenizer.get_words(preprocessing(line.lower()))
        # if len(tokens) == 0:
        #     continue
        result.append(tokens)

    return result

In [ ]:
 corpus_df = [id2word_corpora.doc2bow(text) for text in preprocess_text(df.articleBody.values)]

In [ ]:
len(corpus_df)

46187

# LDA model

In [ ]:
from gensim.models import LdaModel, LdaMulticore
from pprint import pprint

In [ ]:
model_lda = LdaMulticore(corpus=corpus,
                            id2word=id2word_corpora,
                            workers=4,
                            num_topics=20,
                            random_state=42,
                            chunksize=100,
                            passes=15,
                            alpha=0.04,
                            eta=0.1,
                            iterations=2000,
                            per_word_topics=True,)

In [ ]:
model_lda4 = LdaMulticore(corpus=corpus,
                            id2word=id2word_corpora,
                            workers=4,
                            num_topics=25,
                            random_state=42,
                            chunksize=100,
                            passes=15,
                            alpha=0.04,
                            eta=0.1,
                            iterations=2000,
                            per_word_topics=True,)

In [ ]:
history = []

for num_t in [8,20,25]:
  for al in [0.05, 0.08, 0.1]:
      model = LdaMulticore(corpus=corpus,
                                id2word=id2word_corpora,
                                workers=5,
                                num_topics=num_t,
                                random_state=42,
                                chunksize=100,
                                passes=15,
                                alpha=al,
                                eta=0.1,
                                iterations=1000,
                                per_word_topics=True,)
      coherence_model_lda = CoherenceModel(model=model, texts=texts_test, dictionary=id2word_corpora, coherence='c_v')
      coherence_lda = coherence_model_lda.get_coherence()
      history.append({"num_topics":num_t, "coherence_lda":coherence_lda, "alpha":al})

In [ ]:
history

[{'alpha': 0.05, 'coherence_lda': 0.4831173653687284, 'num_topics': 8},
 {'alpha': 0.08, 'coherence_lda': 0.4918668494628131, 'num_topics': 8},
 {'alpha': 0.1, 'coherence_lda': 0.484270345447888, 'num_topics': 8},
 {'alpha': 0.05, 'coherence_lda': 0.5601525438301194, 'num_topics': 20},
 {'alpha': 0.08, 'coherence_lda': 0.5586349511365969, 'num_topics': 20},
 {'alpha': 0.1, 'coherence_lda': 0.5531695097479302, 'num_topics': 20},
 {'alpha': 0.05, 'coherence_lda': 0.584669378651952, 'num_topics': 25},
 {'alpha': 0.08, 'coherence_lda': 0.5788300094961877, 'num_topics': 25},
 {'alpha': 0.1, 'coherence_lda': 0.5924430213821378, 'num_topics': 25}]

In [ ]:
input()

In [ ]:
import requests

headers = dict()
headers["Accept"] ="*/*"
headers["Host"] = "clients5.google.com"
headers["Connection"] =  "keep-alive"
headers["Upgrade-Insecure-Requests"] = "1"
headers["User-Agent"] = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36"
headers["Content-Encoding"] = "gzip"
headers["Accept-Language"] = "en-US,en;q=0.9,ru;q=0.8,he;q=0.7"
headers["Content-Type"] = "application/json; charset=UTF-8"

def translate_text(text, lang='en'):
    url = f"https://clients5.google.com/translate_a/t?client=dict-chrome-ex&sl=auto&tl={lang}&q={text}"
    response = requests.get(url, verify=True, headers=headers)
    response.encoding = response.apparent_encoding
    json = response.json()
    try:
        return "\n".join(map(lambda p: p['trans'], json['sentences']))
    except:
        pass
    return text

In [ ]:
cls2id

{'criminal': 7,
 'entertainment': 5,
 'food': 6,
 'health': 4,
 'politics': 3,
 'sports': 1,
 'tech': 2}

In [ ]:
model_lda.log_perplexity

<bound method LdaModel.log_perplexity of <gensim.models.ldamulticore.LdaMulticore object at 0x7f954e729a90>>

In [ ]:
df.label_new.unique()

array(['sports', 'tech', 'politics', 'health', 'entertainment', 'food',
       'criminal'], dtype=object)

In [ ]:
p = []
for v in corpus_df:
  dist = model_lda.get_document_topics(v)
  m = np.array(list(map(lambda t:t[1], dist))).max()
  p.append(claster_map_lda[list(filter(lambda t: np.isclose(t[1], m), dist))[0][0]])

In [ ]:
train_vecs = []
for i in range(len(corpus_df)):
    top_topics = model_lda.get_document_topics(corpus_df[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(20)]
    # topic_vec.extend([rev_train.iloc[i].real_counts]) # counts of reviews for restaurant
    # topic_vec.extend([len(rev_train.iloc[i].text)]) # length review
    train_vecs.append(topic_vec)

In [ ]:
X = np.array(train_vecs)
y = df.label.map(cls2id).values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(XXX, y, test_size=0.2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
cls = RandomForestClassifier().fit(X_train, y_train)

In [ ]:
y_pred = cls.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.77      0.52      0.62        92
           2       0.93      0.93      0.93      3885
           3       0.81      0.83      0.82      1763
           4       0.85      0.89      0.87      2877
           5       0.89      0.79      0.84       430
           6       0.88      0.78      0.83        46
           7       0.80      0.30      0.43       145

    accuracy                           0.88      9238
   macro avg       0.85      0.72      0.76      9238
weighted avg       0.88      0.88      0.88      9238



In [ ]:
cls = RandomForestClassifier().fit(X_train, y_train)

In [ ]:
y_pred = cls.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.90      0.48      0.63       118
           2       0.92      0.95      0.93      3857
           3       0.87      0.84      0.85      1771
           4       0.86      0.91      0.89      2825
           5       0.94      0.82      0.88       474
           6       0.87      0.68      0.76        57
           7       0.93      0.58      0.71       136

    accuracy                           0.89      9238
   macro avg       0.90      0.75      0.81      9238
weighted avg       0.90      0.89      0.89      9238



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_fasttext, y, test_size=0.2)

In [ ]:
cls = RandomForestClassifier().fit(X_train, y_train)

In [ ]:
y_pred = cls.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.86      0.49      0.62       100
           2       0.93      0.93      0.93      3899
           3       0.85      0.82      0.84      1751
           4       0.85      0.91      0.88      2870
           5       0.94      0.82      0.88       445
           6       0.88      0.80      0.84        56
           7       0.86      0.46      0.60       117

    accuracy                           0.89      9238
   macro avg       0.88      0.75      0.80      9238
weighted avg       0.89      0.89      0.89      9238



In [ ]:
from gensim.models import CoherenceModel

coherence_model_lda = CoherenceModel(model=model_lda, texts=texts_train, dictionary=id2word_corpora, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5677295431070475


In [ ]:
from gensim.models import CoherenceModel

coherence_model_lda = CoherenceModel(model=model_lda, texts=texts_test, dictionary=id2word_corpora, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5544586584302917


In [ ]:
from gensim.models import CoherenceModel

coherence_model_lda = CoherenceModel(model=model_lda2, texts=texts_test, dictionary=id2word_corpora, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5271926772934189


In [ ]:
from gensim.models import CoherenceModel

coherence_model_lda = CoherenceModel(model=model_lda3, texts=texts_test, dictionary=id2word_corpora, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5454024858760793


In [ ]:
!cp -r /content/huawei-nlpcourse-project /content/drive/MyDrive/hebert

In [ ]:
!mkdir -p model_lda3

In [ ]:
model_lda3.save("model_lda3/lda")

In [ ]:
from gensim.models import CoherenceModel

coherence_model_lda = CoherenceModel(model=model_lda3, texts=corpus, dictionary=id2word_corpora, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/usr/local/lib/python3.6/dist-packages/gensim/topic_coherence/direct_confirmation_measure.py:193: RuntimeWarning: invalid value encountered in true_divide
  numerator = (co_occur_count / num_docs) + EPSILON
/usr/local/lib/python3.6/dist-packages/gensim/topic_coherence/direct_confirmation_measure.py:194: RuntimeWarning: invalid value encountered in true_divide
  denominator = (w_prime_count / num_docs) * (w_star_count / num_docs)
/usr/local/lib/python3.6/dist-packages/gensim/topic_coherence/direct_confirmation_measure.py:189: RuntimeWarning: invalid value encountered in true_divide
  co_doc_prob = co_occur_count / num_docs



Coherence Score:  nan


In [ ]:
from gensim.models import CoherenceModel

coherence_model_lda = CoherenceModel(model=model_lda3, texts=corpus, dictionary=id2word_corpora, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
l = np.array(df2.articleBody.str.count(" ")+1)

In [ ]:
df.sample(2)

,Unnamed: 0,articleBody,description,headline,keywords,title,type,category,source,title_eng,articleBodyLen,datePublished,subtitle,description_eng,label_zero_shot,label_finetuned,label,label_new,row_num
91227,91227,"אין רשומות לתצוגה\nיום חמישי, 10/4/14, 23:36\n...",צביקה נדלר,צביקה נדלר,"אוכל ויין, אינטרנט ומחשבים, מוסיקה, ספורט, עיצ...",צביקה נדלר,other,entertainment,themarker,Zvika Nadler,NaN,NaN,NaN,NaN,NaN,health,entertainment,entertainment,91227
30892,30892,"לפחות 40 בני אדם נהרגו, בהם 11 ילדים, בפיצוץ ב...",הארגון הסורי לזכויות אדם דיווח כי שישה לוחמים ...,סוריה: יותר מ-40 הרוגים בפיצוץ משאית תופת סמוך...,"['טורקיה', 'סוריה', 'מטען חבלה', 'פיצוץ', 'אפר...",סוריה: יותר מ-40 הרוגים בפיצוץ משאית תופת סמוך...,article,politics,3,Syria: More than 40 people killed in a car bom...,431.0,NaN,NaN,The Syrian Observatory for Human Rights report...,politics,politics,politics,politics,30892


$U ∈ \mathbb{R}^{m×d_{BERT}}$ 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_labse, y, test_size=0.2)

In [ ]:
cls = RandomForestClassifier().fit(X_train, y_train)

In [ ]:
y_pred = cls.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.86      0.12      0.21       100
           2       0.90      0.94      0.92      3825
           3       0.87      0.85      0.86      1757
           4       0.86      0.91      0.88      2925
           5       0.94      0.78      0.85       451
           6       0.88      0.13      0.23        54
           7       0.91      0.40      0.55       126

    accuracy                           0.88      9238
   macro avg       0.89      0.59      0.64      9238
weighted avg       0.88      0.88      0.88      9238



In [ ]:
from utils.utils import from_h5

In [ ]:
pd.DataFrame(emb[:10_000]).to_csv("v.tsv", sep='\t', index=False)

In [ ]:
df_train["label"][:10_000].to_csv("w.tsv", sep='\t', index=False)

In [ ]:
from google.colab import files

files.download('v.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
emb = from_h5("/content/drive/MyDrive/hebert/embeddings_summary.h5")

In [ ]:
emb_fasttext = from_h5("/content/drive/MyDrive/hebert/fasstext_text_mean.h5")

In [ ]:
emb_labse = from_h5("/content/drive/MyDrive/hebert/desc_labse_embeddings.h5")

In [ ]:
emb_labse = emb_labse[df.row_num.values]

In [ ]:
emb_fasttext = emb_fasttext[df.row_num.values]

In [ ]:
X_fasttext = np.hstack([X, emb_fasttext])

In [ ]:
X_labse = np.hstack([X, emb_labse])

In [ ]:
len(emb)

164606

In [ ]:
len(X)

46187

In [ ]:
XX=emb[df.row_num.values]

In [ ]:
XXX = np.hstack([X,XX])

In [ ]:
import hashlib

text2ind = dict()

for i, row in df2.iterrows():
  text2ind[hashlib.sha1(row.articleBody.encode('utf-8')).hexdigest()] = i

In [ ]:
df = df[~df.label_new.isna()]

In [ ]:
len(corpus_test)+len(corpus)

46087

In [ ]:
model_lda.get_document_topics(corpus[87])

[(1, 0.10147892),
 (3, 0.043906517),
 (6, 0.74534076),
 (8, 0.030509664),
 (10, 0.011151256),
 (11, 0.016975366),
 (16, 0.048344757)]

In [ ]:
train_vecs = []
for i in range(len(rev_train)):
    top_topics = lda_train.get_document_topics(train_corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(20)]
    topic_vec.extend([rev_train.iloc[i].real_counts]) # counts of reviews for restaurant
    topic_vec.extend([len(rev_train.iloc[i].text)]) # length review
    train_vecs.append(topic_vec)

In [ ]:
claster_map_lda = {0:'business', 1:'health', 2:'politics', 3:'entertainment', 4:'economics', 5:'health', 
                   6:'politics', 7:'politics', 8:'tech', 9:'food', 10:'health', 11:'economics', 12:'unk',
                   13:'tech', 14:'economics', 15:'entertainment', 16:'unk', 17:'politics', 18:'unk', 19:'economics'}

In [ ]:
 Sports, Tech, Politics, Economics, Health, Criminal, Entertainment, Food, business

In [ ]:
pprint([translate_text(t) for t in model_lda.print_topics()])

['(0, \'0.015 * "Work" 0.013 * "Work" 0.010 * "Experience" 0.006 * "Life" '
 '0.004 * "Find" 0.004 * "Develop" 0.004 * "Work" 0.003 * "Work" 0.003 * '
 '"Advantage" 0.003 \n'
 '* "Jobs")',
 '(1, \'0.021 * "Health" 0.019 * "Corona" 0.014 * "Patients" 0.011 * "Office" '
 '0.008 * "Today" 0.007 * "Patients" 0.007 * "Corona" 0.005 * "At home" 0.005 '
 '* "After" 0.005 \n'
 '*"said"\')',
 '(2, \'0.025 * "Netanyahu" 0.023 * "Government" 0.016 * "Head" 0.008 * '
 '"White" 0.008 * "Ganz" 0.007 * "Benjamin" 0.007 * "Law" 0.007 * "Knesset" '
 '0.007 * "Today" 0.007 \n'
 '*"Israel"\')',
 '(3, \'0.011 * "at home" 0.010 * "children" 0.009 * "school" 0.008 * '
 '"education" 0.008 * "home" 0.007 * "children" 0.006 * "son" 0.005 * "today" '
 '0.005 * "women" 0.004 \n'
 '*"a girl"\')',
 '(4, \'0.010 * "million" 0.009 * "dollars" 0.007 * "company" 0.007 * '
 '"billion" 0.007 * "most" 0.006 * "this year" 0.006 * "times" 0.004 * "per '
 'year" 0.004 * "last" 0.004 \n'
 '*"already"\')',
 '(5, \'0.006 * "Tr

In [ ]:
!mkdir -p lda

In [ ]:
model_lda.save('lda/lda.dump')

In [ ]:
model_lda2 = LdaMulticore(corpus=corpus,
                            id2word=id2word_corpora,
                            workers=4,
                            num_topics=12,
                            random_state=42,
                            chunksize=100,
                            passes=15,
                            alpha=0.06,
                            eta=0.1,
                            iterations=1000,
                            per_word_topics=True,)

In [ ]:
pprint(model_lda2.print_topics())

[(0,
  '0.010*"החברה" + 0.006*"ידי" + 0.005*"מידע" + 0.004*"כדי" + 0.004*"המערכת" + '
  '0.004*"מערכת" + 0.003*"המידע" + 0.003*"גיוס" + 0.003*"באופן" + '
  '0.003*"חברת"'),
 (1,
  '0.021*"הקורונה" + 0.017*"הבריאות" + 0.013*"החולים" + 0.011*"היום" + '
  '0.011*"בני" + 0.010*"אדם" + 0.010*"משרד" + 0.009*"חולים" + 0.008*"מספר" + '
  '0.007*"בנגיף"'),
 (2,
  '0.021*"הממשלה" + 0.020*"נתניהו" + 0.014*"ראש" + 0.008*"ישראל" + '
  '0.007*"היום" + 0.006*"לבן" + 0.006*"גנץ" + 0.006*"בנימין" + 0.006*"יו" + '
  '0.006*"הכנסת"'),
 (3,
  '0.007*"אחד" + 0.006*"הזה" + 0.005*"הרבה" + 0.005*"שלא" + 0.005*"כמה" + '
  '0.004*"צריך" + 0.004*"אנשים" + 0.004*"שהוא" + 0.004*"לעשות" + 0.004*"ולא"'),
 (4,
  '0.016*"החברה" + 0.012*"דולר" + 0.011*"מיליון" + 0.008*"חברות" + '
  '0.006*"חברת" + 0.005*"בישראל" + 0.005*"בעולם" + 0.004*"בתחום" + '
  '0.004*"ביותר" + 0.004*"מיליארד"'),
 (5,
  '0.008*"כדי" + 0.007*"חמאס" + 0.006*"ישראל" + 0.005*"הרש" + 0.005*"עזה" + '
  '0.004*"הארץ" + 0.004*"צה" + 0.004*"האוויר" + 0.003

In [ ]:
model_lda3 = LdaMulticore(corpus=corpus,
                            id2word=id2word_corpora,
                            workers=4,
                            num_topics=15,
                            random_state=42,
                            chunksize=100,
                            passes=15,
                            alpha=0.05,
                            eta=0.2,
                            iterations=2000,
                            per_word_topics=True,)

In [ ]:
pprint(model_lda3.print_topics())

[(0,
  '0.022*"החברה" + 0.009*"מיליון" + 0.008*"דולר" + 0.008*"ידי" + '
  '0.006*"המערכת" + 0.006*"גיוס" + 0.005*"חברת" + 0.005*"סבב" + '
  '0.005*"הישראלית" + 0.004*"כדי"'),
 (1,
  '0.022*"הקורונה" + 0.020*"הבריאות" + 0.016*"החולים" + 0.011*"בני" + '
  '0.010*"אדם" + 0.010*"היום" + 0.010*"משרד" + 0.010*"חולים" + 0.008*"בנגיף" + '
  '0.008*"מספר"'),
 (2,
  '0.020*"הממשלה" + 0.019*"נתניהו" + 0.012*"ראש" + 0.006*"היום" + '
  '0.006*"ישראל" + 0.006*"גנץ" + 0.006*"לבן" + 0.006*"הכנסת" + 0.006*"בנימין" '
  '+ 0.006*"אמר"'),
 (3,
  '0.006*"בן" + 0.006*"ילדים" + 0.005*"בבית" + 0.005*"נשים" + 0.004*"הילדים" + '
  '0.004*"היום" + 0.004*"בת" + 0.004*"חיים" + 0.003*"בית" + 0.003*"אביב"'),
 (4,
  '0.013*"החברה" + 0.012*"דולר" + 0.012*"מיליון" + 0.009*"חברות" + '
  '0.006*"מיליארד" + 0.005*"בישראל" + 0.005*"חברת" + 0.004*"ביותר" + '
  '0.004*"בעולם" + 0.004*"השנה"'),
 (5,
  '0.012*"חמאס" + 0.011*"ישראל" + 0.009*"הרש" + 0.008*"עזה" + 0.007*"צה" + '
  '0.006*"כדי" + 0.006*"הביטחון" + 0.005*"יום" + 0.

In [ ]:
predicted_topics_per_doc = [lda_model_my.get_document_topics(text) for text in corpus_test]
predicted_aspects_lda = [max(pair, key=lambda x:x[1])[0] for pair in  predicted_topics_per_doc]

In [ ]:
claster_map_lda = {0:'Staff', 1:'Ambience', 2:'Miscellaneous', 3:'Miscellaneous', 4:'Food', 5:'Ambience', 
                   6:'Staff', 7:'Food', 8:'Food', 9:'Ambience', 10:'Food', 11:'Staff', 12:'Ambience',
                   13:'Miscellaneous'}

In [ ]:
cls2id

{'criminal': 7,
 'entertainment': 5,
 'food': 6,
 'health': 4,
 'politics': 3,
 'sports': 1,
 'tech': 2}

In [ ]:
predicted_labels_lda = [claster_map_lda[i] for i in predicted_aspects_lda]
compare_lda_to_test = [mapper[label] for label in predicted_labels_lda]

In [ ]:
print(classification_report(encoded_test_labels, compare_lda_to_test, target_names=['Food', 'Staff','Ambience',
                                                                                    'Miscellaneous'
                                                                                   ], digits=3))

               precision    recall  f1-score   support

         Food      0.898     0.475     0.621       887
        Staff      0.506     0.494     0.500       352
     Ambience      0.249     0.430     0.315       251
Miscellaneous      0.000     0.000     0.000         0

     accuracy                          0.472      1490
    macro avg      0.413     0.350     0.359      1490
 weighted avg      0.696     0.472     0.541      1490



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
